# Train TLC Trip Demands Predicator

In this notebook, we will train a model to predict the trip demands in each region. We will use all historical data and AutoGluon to 
train the model. And predict the next three 24 hours trip demands in all NYC regions.


# Prerequisites

We use Amazon SageMaker to train the model. Make sure you have an SageMaker exectuion role, and have the permissions to run training jobs, and load data. 

In [1]:
# Define it is a production environment or sampled environment
PROD=False

In [2]:
import sagemaker
import pandas as pd
from ag_model import (
    AutoGluonSagemakerEstimator,
    AutoGluonNonRepackInferenceModel,
    AutoGluonSagemakerInferenceModel,
    AutoGluonRealtimePredictor,
    AutoGluonBatchPredictor,
)
from sagemaker import utils
from sagemaker.serializers import CSVSerializer
import os
import boto3

role = "arn:aws:iam::178770047227:role/service-role/SageMaker-ExecutionRole-20231202T212840" # change to your role
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session._region_name

bucket = "qiaoshi-aws-ml"   # change to your role
s3_prefix = f"tlc/ml/{utils.sagemaker_timestamp()}"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/qiaoshi/Library/Application Support/sagemaker/config.yaml


# Training

Users can create their own training/inference scripts using [SageMaker Python SDK examples](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script).
The scripts we created allow to pass AutoGluon configuration as a YAML file (located in `data/config` directory).

We are using [official AutoGluon Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers) with custom training scripts (see `scripts/` directory).

In [ ]:
ag = AutoGluonSagemakerEstimator(
    role=role,
    entry_point="scripts/tabular_train.py",
    region=region,
    instance_count=1,
    instance_type="ml.p3.8xlarge" if PROD else "ml.m5.2xlarge",
    framework_version="0.8.2",
    py_version="py39",
    base_job_name="tlc-tabular-train",
    disable_profiler=True,
    debugger_hook_config=False,
)

Upload the scripts to s3

In [ ]:
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"

train_input = "s3://qiaoshi-aws-ml/tlc/results/ml/trips_with_weather_merged/train.csv" if PROD else "s3://qiaoshi-aws-ml/tlc/results/ml/trips_with_weather_sampled/train.csv"

eval_input = "s3://qiaoshi-aws-ml/tlc/results/ml/trips_with_weather_merged/eval.csv" if PROD else "s3://qiaoshi-aws-ml/tlc/results/ml/trips_with_weather_sampled/eval.csv"

config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-med.yaml"), key_prefix=s3_prefix
)

# Provide inference script so the script repacking is not needed later
# See more here: https://docs.aws.amazon.com/sagemaker/latest/dg/mlopsfaq.html
# Q. Why do I see a repack step in my SageMaker pipeline?
inference_script = ag.sagemaker_session.upload_data(
    path=os.path.join("scripts", "tabular_serve.py"), key_prefix=s3_prefix
)

In [ ]:
train_input

In [ ]:
eval_input

### Fit The Model
For local training set `instance_type` to local.

For non-local training the recommended instance type is `ml.m5.2xlarge`.

In [ ]:
job_name = utils.unique_name_from_base("tlc-training")
ag.fit(
    {
        "config": config_input,
        "train": train_input,
        "test": eval_input,
        "serving": inference_script,
    },
    job_name=job_name,
)

### Model export

AutoGluon models are portable: everything needed to deploy a trained model is in the tarball created by SageMaker.

The artifact can be used locally, on EC2/ECS/EKS or served via SageMaker Inference.

In [ ]:
!aws s3 cp {ag.model_data} .

In [ ]:
!ls -alF model.tar.gz

# Endpoint Deployment

Upload the model we trained earlier

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base("sagemaker-autogluon-serving-trained-model")

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [3]:
model_data = "s3://sagemaker-us-east-1-178770047227/tlc-training-1705471847-f252/output/model.tar.gz"

Deploy remote or local endpoint

In [4]:
instance_type = "ml.m5.2xlarge"

In [5]:
model = AutoGluonNonRepackInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.8.2",
    py_version="py39",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="tabular_serve.py",
)

In [6]:
model.deploy(initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type)

TypeError: AutoGluonNonRepackInferenceModel.prepare_container_def() got an unexpected keyword argument 'accept_eula'

In [ ]:
predictor = AutoGluonRealtimePredictor(model.endpoint_name)

### Predict on unlabeled test data

Remove target variable (`class`) from the data and get predictions for a sample of 100 rows using the deployed endpoint.

In [ ]:
df = pd.read_csv("data/test.csv")
data = df[:100]

In [ ]:
preds = predictor.predict(data.drop(columns="total_trips"))
preds

In [ ]:
p = preds[["pred"]]
p = p.join(data["class"]).rename(columns={"class": "actual"})
p.head()

In [ ]:
print(f"{(p.pred==p.actual).astype(int).sum()}/{len(p)} are correct")

### Cleanup Endpoint

In [ ]:
predictor.delete_endpoint()

# Batch Transform

Deploying a trained model to a hosted endpoint has been available in SageMaker since launch and is a great way to provide real-time predictions to a service like a website or mobile app. But, if the goal is to generate predictions from a trained model on a large dataset where minimizing latency isn’t a concern, then the batch transform functionality may be easier, more scalable, and more appropriate.

[Read more about Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base(
    "sagemaker-autogluon-batch_transform-trained-model"
)

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [ ]:
instance_type = "ml.m5.2xlarge"

In [ ]:
model = AutoGluonSagemakerInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.6",
    py_version="py38",
    instance_type=instance_type,
    entry_point="tabular_serve-batch.py",
    source_dir="scripts",
    predictor_cls=AutoGluonBatchPredictor,
)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=instance_type,
    strategy="MultiRecord",
    max_payload=6,
    max_concurrent_transforms=1,
    output_path=output_path,
    accept="application/json",
    assemble_with="Line",
)

Prepare data for batch transform

In [ ]:
pd.read_csv(f"data/test.csv")[:100].to_csv("data/test_no_header.csv", header=False, index=False)

Upload data to sagemaker session

In [ ]:
test_input = transformer.sagemaker_session.upload_data(
    path=os.path.join("data", "test_no_header.csv"), key_prefix=s3_prefix
)

In [ ]:
transformer.transform(
    test_input,
    input_filter="$[:14]",  # filter-out target variable
    split_type="Line",
    content_type="text/csv",
    output_filter="$['class']",  # keep only prediction class in the output
)

transformer.wait()

Download batch transform outputs

In [ ]:
!aws s3 cp {transformer.output_path[:-1]}/test_no_header.csv.out .

In [ ]:
p = pd.concat(
    [
        pd.read_json("test_no_header.csv.out", orient="index")
        .sort_index()
        .rename(columns={0: "preds"}),
        pd.read_csv("data/test.csv")[["class"]].iloc[:100].rename(columns={"class": "actual"}),
    ],
    axis=1,
)
p.head()

In [ ]:
print(f"{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct")

# Conclusion

In this tutorial we successfully trained an AutoGluon model and explored a few options how to deploy it using SageMaker. Any of the sections of this tutorial (training/endpoint inference/batch inference) can be used independently (i.e. train locally, deploy to SageMaker, or vice versa).

Next steps:
* [Learn more](https://auto.gluon.ai) about AutoGluon, explore [tutorials](https://auto.gluon.ai/stable/tutorials/index.html).
* Explore [SageMaker inference documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).